In [7]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import vtk
from vtkmodules.util import numpy_support
from data.data import fitTransPipeline
from cfg.configs import Config
from train import load_checkpoint

In [2]:
data_path = "../test/final/data/bc80-45000-down.vts"
vtkSGR = vtk.vtkXMLStructuredGridReader()
vtkSGR.SetFileName(data_path)
vtkSGR.Update()
vtkSG = vtkSGR.GetOutput()
print(vtkSG.GetDimensions())
print(vtkSG.GetExtent())
print(vtkSG.GetBounds())

(88, 51, 21)
(0, 87, 0, 50, 0, 20)
(0.0, 696.0, -250.0, 150.0, 0.7504535071217414, 429.4370158281313)


In [8]:
dims = [21, 51, 88]
z,y,x = np.meshgrid(
  np.linspace(0, 1, dims[0]),
  np.linspace(0, 1, dims[1]),
  np.linspace(0, 1, dims[2]),
  indexing="ij"
)
xyz = np.concatenate([z[...,None], y[...,None], x[...,None]], axis=-1)

In [3]:
class Phys2CompDataset(Dataset):
  def __init__(self, data_path, intrans=None, outtrans=None):
    self.data_path = data_path
    vtkSGR = vtk.vtkXMLStructuredGridReader()
    vtkSGR.SetFileName(data_path)
    vtkSGR.Update()
    vtkSG = vtkSGR.GetOutput()
    # get dimension and swap xyz order to zyx
    self.dims = np.array(vtkSG.GetDimensions())
    tmp = self.dims[0]
    self.dims[0] = self.dims[2]
    self.dims[2] = tmp
    # get phys mesh
    self.phys = numpy_support.vtk_to_numpy(vtkSG.GetPoints().GetData())
    # get comp mesh - regular grid
    self.phys = self.phys.reshape([*self.dims, 3])
    z,y,x = np.meshgrid(
      np.linspace(0, 1, self.dims[0]),
      np.linspace(0, 1, self.dims[1]),
      np.linspace(0, 1, self.dims[2]),
      indexing="ij"
    )
    self.comp = np.concatenate([z[...,None], y[...,None], x[...,None]], axis=-1)
    # preprocessing
    self.phys_prep = torch.Tensor(self.phys.reshape(-1, 3))
    self.comp_prep = torch.Tensor(self.comp.reshape(-1, 3))

    if intrans is not None:
      print("transforming inputs")
      self.phys_prep, self.inpp = fitTransPipeline(self.phys.reshape(-1, 3))
      self.phys_prep = torch.Tensor(self.phys_prep)
    
    if outtrans is not None:
      print("transforming outputs")
      self.comp_prep, self.outpp = fitTransPipeline(self.comp.reshape(-1, 3))
      self.comp_prep = torch.Tensor(self.comp_prep)

  def __len__(self):
    return len(self.phys_prep)

  def __getitem__(self, idx):
    return self.phys_prep[idx], self.comp_prep[idx]
    
ds = Phys2CompDataset(data_path)

NameError: name 'dims' is not defined

In [8]:
nncfg = Config("nn_vls_md.yaml")

In [9]:
model_dir = nncfg.config['model']['log_dir']
ckpt = load_checkpoint(f"{model_dir}/ckpt_2499_-001_2.15e-02", nncfg)
model = ckpt['model']

In [12]:
ds = nncfg.get_dataset()

In [17]:
pred = model(ds.phys_prep)

In [19]:
for i in range(10):
  print(ds.comp_prep[i], pred[i])

tensor([0., 0., 0.]) tensor([0.2683, 0.0543, 0.2750], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0115]) tensor([0.0579, 0.0965, 0.0503], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0230]) tensor([0.0152, 0.1133, 0.0966], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0345]) tensor([0.1242, 0.1006, 0.1399], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0460]) tensor([0.0835, 0.0642, 0.1862], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0575]) tensor([0.2448, 0.1216, 0.3506], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0690]) tensor([0.1515, 0.0638, 0.1328], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0805]) tensor([0.0130, 0.1493, 0.1136], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0920]) tensor([0.0103, 0.1504, 0.1529], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.1034]) tensor([0.0101, 0.0926, 0.0751], grad_fn=<SelectBackward0>)
